<a href="https://colab.research.google.com/github/linhkid/Google-IO-Extended-speechs/blob/main/notebooks/Gemma_2b_and_7b_Google_I_O_2024_Extended.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Introduction

Google recently released a series of open source LLMs based on their Gemini flagship model. These smaller models are built using the same research and training methodologies that Google used to create Gemini and come with very big promises in how they will reshape the open source LLM space. The models come in 2 sizes, 2B and 7B making them small enough that they can sit on consumer level hardware, and even Google's own Collab service if users don't want or don't have access to their own personal GPUs. The Gemma models are exciting entries into the LLM race and I'm excited to explore them. In this notebook I'll go over how to access these models and run them in your own environment using Huggingface's libraries and tools.

# Preparation

In [1]:
!pip install -q --upgrade transformers accelerate bitsandbytes flash_attn accelerate datasets peft

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 309.4/309.4 kB 7.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.8/119.8 MB 13.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 92.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 52.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 251.6/251.6 kB 1.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.2/43.2 kB 7.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.8/40.8 MB 41.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 20.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 11.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 28.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 23.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21.3 MB 

In [1]:
import os
from google.colab import userdata
os.environ["HF_TOKEN"] = userdata.get('HF_TOKEN')


# Gemma 2B

Introduction about Gemma 2b here


## Original FP (torch.float32)


In [3]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

MODEL_NAME = "google/gemma-2b-it"
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    device_map="auto"
)

input_text = "Write me a poem about Google I/O event."
input_ids = tokenizer(input_text, return_tensors="pt").to("cuda")

outputs = model.generate(**input_ids, max_new_tokens=256)
print(tokenizer.decode(outputs[0]))

tokenizer_config.json:   0%|          | 0.00/34.2k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/636 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/627 [00:00<?, ?B/s]

...

model-00002-of-00002.safetensors:   0%|          | 0.00/67.1M [00:00<?, ?B/s]

`config.hidden_act` is ignored, you should use `config.hidden_activation` instead.
Gemma's activation function will be set to `gelu_pytorch_tanh`. Please, use
`config.hidden_activation` if you want to override this behaviour.
See https://github.com/huggingface/transformers/pull/29402 for more details.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/137 [00:00<?, ?B/s]

<bos>Write me a poem about Google I/O event.

A digital stage, a virtual space,
Where ideas take their place.
From startups to giants, a diverse array,
Sharing stories, insights, and ways.

The lights shine bright, the speakers speak,
A symphony of knowledge, a vibrant streak.
From AI to VR, the topics unfold,
Connecting minds, stories to be told.

With every click, a new adventure unfolds,
A world of possibilities, a story to be told.
Google I/O, a beacon in the night,
Guiding the future, shining ever bright.<eos>


## Using BFloat16


In [4]:
model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    device_map="auto",
    torch_dtype=torch.bfloat16
)

# input_text = "Write me a poem about Google I/O event."
# input_ids = tokenizer(input_text, return_tensors="pt").to("cuda" )

outputs = model.generate(**input_ids,  max_new_tokens=256)
print(tokenizer.decode(outputs[0]))



Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

<bos>Write me a poem about Google I/O event.

A digital stage, a virtual space,
Where ideas take their place.
From startups to giants, a diverse array,
Sharing stories, insights, and ways.

The lights shine bright, the speakers speak,
A symphony of knowledge, a vibrant streak.
From AI to VR, the topics unfold,
Connecting minds, stories to be told.

With every click, a new adventure unfolds,
A world of possibilities, a story to be told.
Google I/O, a beacon in the night,
Guiding the future, shining ever bright.<eos>


## Quantized 8-bit Integer

In [5]:
from transformers import BitsAndBytesConfig

quant_config = BitsAndBytesConfig(load_in_8bit=True)

model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    quantization_config=quant_config
)

outputs = model.generate(**input_ids,  max_new_tokens=256)
print(tokenizer.decode(outputs[0]))

`low_cpu_mem_usage` was None, now set to True since model is quantized.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

<bos>Write me a poem about Google I/O event.

A digital stage, a virtual space,
Where ideas take their place.
From startups to giants, a diverse array,
Sharing their stories, come what may.

With presentations, panels, and talks,
A chance to learn, to grow, to recall.
The energy's electric, the atmosphere's alive,
As the world's brightest minds connect and thrive.

Google I/O, a beacon of light,
Guiding the future, shining bright.
A platform for collaboration, a stage for debate,
Where the power of ideas can't be beat.

So let us join the virtual throng,
And immerse ourselves in this digital throng.
Google I/O, a symphony of thought,
A testament to the power of our youth.<eos>


## Quantized 4-bit precision


In [6]:
quant_config = BitsAndBytesConfig(load_in_4bit=True,
              bnb_4bit_use_double_quant=True,
              bnb_4bit_quant_type="nf4",
              bnb_4bit_compute_dtype=torch.float16)

model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    quantization_config=quant_config
)

outputs = model.generate(**input_ids,  max_new_tokens=256)
print(tokenizer.decode(outputs[0]))

`low_cpu_mem_usage` was None, now set to True since model is quantized.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

<bos>Write me a poem about Google I/O event.

A digital stage, a virtual space,
Where ideas flow, a boundless chase.
From startups to giants, a diverse array,
Sharing stories, insights, and a brighter day.

With every click, a journey takes flight,
From product launches to data's might.
The energy is electric, the atmosphere alive,
As communities gather, a collective thrive.

With every session, a spark is ignited,
A thirst for knowledge, a hunger to ignite.
From AI to marketing, the topics unfold,
A symphony of innovation, a story to be told.

So let us gather, in this digital space,
To learn, to grow, and to leave our trace.
Google I/O, a beacon of hope and light,
A journey that inspires, day and night.<eos>


## Flash Attention

In [7]:
model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    torch_dtype=torch.float16,
    attn_implementation="flash_attention_2"
).to(0)

outputs = model.generate(**input_ids,  max_new_tokens=256)
print(tokenizer.decode(outputs[0]))

You are attempting to use Flash Attention 2.0 with a model not initialized on GPU. Make sure to move the model to GPU after initializing it on CPU with `model.to('cuda')`.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

<bos>Write me a poem about Google I/O event.

A digital stage, a virtual space,
Where ideas take their place.
From startups to giants, a diverse array,
Sharing stories, insights, and ways.

The lights shine bright, the speakers speak,
A symphony of knowledge, a vibrant streak.
From AI to VR, the topics unfold,
Connecting minds, stories to be told.

With every click, a new adventure unfolds,
A world of possibilities, a story to be told.
Google I/O, a beacon in the night,
Guiding the future, shining ever bright.<eos>


# Gemma 7B

## Original FP (torch.float32)

In [2]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

MODEL_NAME = "google/gemma-7b-it"
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    device_map="auto"
)

input_text = "Write me a poem about Google I/O event."
input_ids = tokenizer(input_text, return_tensors="pt").to("cuda")

outputs = model.generate(**input_ids, max_new_tokens=256)
print(tokenizer.decode(outputs[0]))

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

KeyboardInterrupt: 

## Using BFloat16

In [1]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

MODEL_NAME = "google/gemma-7b-it"
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    device_map="auto",
    torch_dtype=torch.bfloat16
)

input_text = "Write me a poem about Google I/O event."
input_ids = tokenizer(input_text, return_tensors="pt").to("cuda" )

outputs = model.generate(**input_ids,  max_new_tokens=256)
print(tokenizer.decode(outputs[0]))



`config.hidden_act` is ignored, you should use `config.hidden_activation` instead.
Gemma's activation function will be set to `gelu_pytorch_tanh`. Please, use
`config.hidden_activation` if you want to override this behaviour.
See https://github.com/huggingface/transformers/pull/29402 for more details.


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

<bos>Write me a poem about Google I/O event.

In the heart of Silicon Valley, a stage unfolds,
Where tech giants gather, stories untold.
Google I/O, a moment of grace,
Where the future takes shape at an unprecedented pace.

With keynote speakers, a captivating start,
They unveil visions, ignite the heart.
Products unveiled, a glimpse of delight,
The latest innovations, shining so bright.

The halls echo with energy and zest,
As developers gather, their spirits crest.
Workshops ignite, ideas take flight,
Building the future with all their might.

From mobile apps to AI, the spectrum expands,
The power of technology in the palm of hands.
With every demo, a new story unfolds,
The potential unleashed, a tale to behold.

The energy is high, the atmosphere charged,
As the community connects, a force unmarred.
In the spirit of innovation, they share their dreams,
Building a future where anything can be seen.

So let us celebrate this day of grace,
Where the tech world comes to its place.
Goog

## Quantized 8-bit Integer

In [9]:
import gc

model.cpu()
del model
gc.collect()
torch.cuda.empty_cache()

In [7]:
from transformers import BitsAndBytesConfig

quant_config = BitsAndBytesConfig(load_in_8bit=True)

model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    quantization_config=quant_config
)

outputs = model.generate(**input_ids,  max_new_tokens=256)
print(tokenizer.decode(outputs[0]))

`low_cpu_mem_usage` was None, now set to True since model is quantized.


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

<bos>Write me a poem about Google I/O event.

In the heart of Silicon Valley, a stage unfolds,
Where tech giants gather, stories untold.
Google I/O, a gathering of minds,
Where innovation takes flight, leaving its binds.

With keynote speakers, a captivating start,
They unveil the future, a digital heart.
Android, Wear OS, and Chrome,
The latest advancements, a breathtaking bloom.

Developers swarm, with passion and zest,
Building apps and tools, at an unprecedented crest.
The halls echo with the hum of code,
As creativity blossoms, a vibrant ode.

From wearable gadgets to AI's grace,
The event showcases the future at an unmatched pace.
With every demo, a new horizon unfolds,
A glimpse into the future, where technology beholds.

So, let us celebrate this day of delight,
Where innovation meets passion, shining light.
Google I/O, a journey of dreams,
Where the future takes shape, it would seem.<eos>


## Quantized 4-bit precision

In [8]:
quant_config = BitsAndBytesConfig(load_in_4bit=True,
              bnb_4bit_use_double_quant=True,
              bnb_4bit_quant_type="nf4",
              bnb_4bit_compute_dtype=torch.float16)

model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    quantization_config=quant_config
)

outputs = model.generate(**input_ids,  max_new_tokens=256)
print(tokenizer.decode(outputs[0]))

`low_cpu_mem_usage` was None, now set to True since model is quantized.


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

<bos>Write me a poem about Google I/O event.

In the halls of Silicon Valley,
A stage lights up with glee,
Google I/O takes flight,
A glimpse into the future bright.

With keynote speakers bold,
And products stories untold,
The audience listens with awe,
As innovation takes a bow.

From Android to AI,
The latest trends take flight,
Developers gather strength,
To build the future with intent.

The halls echo with a buzz,
As attendees mingle and discuss,
Ideas spark, connections bloom,
And the spirit of innovation blooms.

So let us celebrate this day,
Where technology finds its way,
To touch our lives in a profound way,
Google I/O, a magical sway.<eos>


## Flash Attention

In [10]:
model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    torch_dtype=torch.float16,
    attn_implementation="flash_attention_2"
).to(0)

outputs = model.generate(**input_ids,  max_new_tokens=256)
print(tokenizer.decode(outputs[0]))

You are attempting to use Flash Attention 2.0 with a model not initialized on GPU. Make sure to move the model to GPU after initializing it on CPU with `model.to('cuda')`.


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

<bos>Write me a poem about Google I/O event.

In the heart of Silicon Valley, a stage unfolds,
Where tech giants gather, stories untold.
Google I/O, a moment of grace,
Where innovation meets a brighter future's embrace.

With keynote speakers, a captivating start,
They unveil visions, ignite the heart.
Products unveiled, a glimpse of delight,
The future of technology, shining so bright.

From Android to Chrome, the latest trends,
Developers gather, their spirits ascend.
With workshops and demos, they learn and grow,
Building apps that will touch and flow.

The halls abuzz with energy and zest,
As attendees connect, their spirits crest.
In the spirit of collaboration, they share their might,
Building a future where technology takes flight.

As the sun sets, the event draws to a close,
Memories made, a lasting glow.
Google I/O, a time for reflection,
Where the future takes shape, beyond imagination.

So let us celebrate this day of grace,
Where innovation meets a brighter place.
May Goog